# EEEN30131 Power System Analysis: Week 02 - Power Flow Formulation

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides an example of the approaches used to develop and simplify the power flow equations, including the proceduer to develop the relevant tools. 

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as you can explore many different variations of the power flow formulation, which will better prepare you for the exams.

## List of contents

- [Load and generation data](#Load-and-generation-data)
- [Developing the power flow equations](#Developing-the-power-flow-equations)
- [Simplifying the equations](#Simplifying-the-equations)
- [Considering known values](#Considering-known-values)
- [Developing as a tool](#Developing-as-a-tool)
- [Case studies](#Case-studies)
  - [Three-bus network](#Three-bus-network)
  - [Two-bus network](#Two-bus-network)
  - [Four-bus network](#Four-bus-network)
  - [Three-bus radial network](#Three-bus-radial-network)
  - [Create your own example](#Create-your-own-example)

## Before we begin

Before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131:
  - Week 1 - Nodal analysis 
  - Week 2 - Power Flow Formulation 
- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

If this data notebook is being used in Jupyter lite, the folders where the python code that supports this notebook are stored, have to be enabled.

In [1]:
from pathlib import Path
if Path.cwd().drive == '':
    a_dir = Path("dir")
    a_dir.mkdir(exist_ok=True)

Some tools have already been developed in other notebooks (e.g., $Y_{bus}$).

In [2]:
from Code.Wk1_EEEN30131 import get_Ybus
import math
import numpy

The tool can now be used using the approach explained in the ***EEEN30131_Week01*** notebook.

In [3]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]
Ybus = get_Ybus(Connectivity, True)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]


[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

## Load and generation data

The $Y_{bus}$ matrix imported above corresponds to the following system.

<img src="Figures/Week02_3Bus.png" alt="Fig01" class="bg-primary" width="500px">

The $Y_{bus}$ matrix only provide the line data, but it does not include information about the loads and generators. This information would have to be added to a new dataset.

The same format used for the connectivity data is used here to represent the loads as we only need to declare the bus where the loads are connected and the relevant active and reactive power.

In [4]:
Load = [
    [2, complex(1.5, 0.8)]
]

The characteristics of the generators also has to be considered for our study. This time we will use python dictionaries to define the generation data, as this can include voltage magnitudes ($V$) and angles ($\Theta$), and active ($P$) and reactive ($Q$) power:
- The voltage magnitude ($V$) has to be defined for slack generators and for generators with voltage control capabilities.
- The voltage angle ($\theta$) has to be defined for slack generators.
- The active power ($P$) has to be defined for generators with and without voltage control capabilities
- The reactive power ($Q$) has to be defined for generators without voltage control capabilities

In [5]:
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1}
]

The first step is to check the information associated with every bus. That is, we need to check which power injections and voltages are known by analysing what is connected to each bus.

In [6]:
def get_Bus_Data(Load, Generator, Ybus):
    Number_Buses = len(Ybus)
    Bus_Data = [{'V': None, '𝜃': None, 'P': 0, 'Q': 0}
                for bus in range(Number_Buses)]

    for load in Load:  # Load can inject active and reactive power
        bus = load[0]-1
        Bus_Data[bus]['P'] -= load[1].real
        Bus_Data[bus]['Q'] -= load[1].imag

    for gen in Generator:  # Generators are a bit more complicated
        if len(gen.keys()) != 3:
            print('Invalid generation data:', gen)
            break
        bus = gen['Bus'] - 1
        # Some generators may inject active and reactive power
        if 'P' in gen.keys() and 'Q' in gen.keys():
            Bus_Data[bus]['P'] += gen['P']
            Bus_Data[bus]['Q'] += gen['Q']
            # Others control voltages
        elif 'P' in gen.keys() and 'V' in gen.keys():
            if Bus_Data[bus]['V'] is not None:
                print('There should only be one generator ', end='')
                print('defining the voltage of bus ', gen['Bus'])
            Bus_Data[bus]['P'] += gen['P']
            Bus_Data[bus]['V'] = gen['V']
            # Others can act as the slack generator
        elif 'V' in gen.keys() and '𝜃' in gen.keys():
            if Bus_Data[bus]['V'] is not None:
                print('There should only be one generator ', end='')
                print('defining the voltage of bus ', gen['Bus'])
            if Bus_Data[bus]['𝜃'] is not None:
                print('There should only be one generator ', end='')
                print('defining the angle of bus ', gen['Bus'])
            Bus_Data[bus]['V'] = gen['V']
            Bus_Data[bus]['𝜃'] = gen['𝜃']
        else:
            print('Invalid generation data:', gen)
            break
    return Bus_Data

Bus_Data_Raw = get_Bus_Data(Load, Generator, Ybus)
print(Bus_Data_Raw)

[{'V': 1, '𝜃': 0, 'P': 0, 'Q': 0}, {'V': None, '𝜃': None, 'P': -1.5, 'Q': -0.8}, {'V': 1, '𝜃': None, 'P': 1, 'Q': 0}]


Now that we have collected the information (i.e., the known and unknown parameters), is is possible to use the data can be used to identify the bus types across the network.

In [7]:
def get_Bus_Type(Ybus, Load, Generator):
    Bus_Data = get_Bus_Data(Load, Generator, Ybus)
    Bus_Type = []
    Number_Buses = len(Ybus)

    for bus in range(Number_Buses):
        # If 𝜃 is known, it has to be a slack bus
        if Bus_Data[bus]['𝜃'] is not None:
            Bus_Type.append(3)
            Bus_Data[bus].pop('P')
            Bus_Data[bus].pop('Q')

            # If its not the slack and we know V, it has to be a PV bus
        elif Bus_Data[bus]['V'] is not None:
            Bus_Type.append(2)
            Bus_Data[bus].pop('Q')
            Bus_Data[bus].pop('𝜃')

            # ALl we have left is PQ buses
        else:
            Bus_Type.append(1)
            Bus_Data[bus].pop('V')
            Bus_Data[bus].pop('𝜃')
    return Bus_Data, Bus_Type

def display_Bus_Type(Bus_Data, Bus_Type):
    print('Bus:  Type:       V:       𝜃:     Pinj:     Qinj:')
    for bus in range(len(Bus_Data)):
        print('%4.0f' % (bus+1), end='')
        if Bus_Type[bus] == 1:
            print('     PQ        ?        ?  %8.4f  %8.4f'
                  % (Bus_Data[bus]['P'], Bus_Data[bus]['Q']))
        elif Bus_Type[bus] == 2:
            print('     PV %8.4f        ?  %8.4f         ?'
                  % (Bus_Data[bus]['V'], Bus_Data[bus]['P']))
        else:
            print('  Slack %8.4f %8.4f         ?         ?'
                  % (Bus_Data[bus]['V'], Bus_Data[bus]['𝜃']))
    
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)
display_Bus_Type(Bus_Data, Bus_Type)

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?


[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

## Developing the power flow equations

We now have enough information to develop the power flow equations. Let us begin with the general formulation of the equations:

$$
P_k = \sum_{i=1}^{N}V_k V_i \left[ G_{k,i} cos (\theta _{k,i}) + B_{k,i} sin (\theta _{k,i})\right]
$$

$$
Q_k = \sum_{i=1}^{N}V_k V_i \left[ G_{k,i} sin (\theta _{k,i}) - B_{k,i} cos (\theta _{k,i})\right]
$$

In order to display the procedure to develop the equations, it is convenient to create a method to help us print the equations (e.g., including subscripts).

In [8]:
def get_string(txt, bus1, bus2=None):
    '''Create string with subscripts'''
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    str1 = str(bus1+1).translate(SUB)
    if bus2 is None:
        str2 = ''
    else:
        str2 = ',' + str(bus2+1).translate(SUB)

    return txt+str1+str2

It can be seen that the equations for the active power and reactive power injections are similar, with only some differences in the trigonometric functions (i.e., `sine` and `cosine`) and signs. Therefore, we create a method to check these differences.

In [9]:
def get_P_Q(txt, No, val=0):
    if txt == 'P':
        if No == 1:
            Str = 'cos'
            Val = math.cos(val)
        elif No == 2:
            Str = '+'
            Val = 1
        elif No == 3:
            Str = 'sin'
            Val = math.sin(val)
    elif txt == 'Q':
        if No == 1:
            Str = 'sin'
            Val = math.sin(val)
        elif No == 2:
            Str = '-'
            Val = -1
        elif No == 3:
            Str = 'cos'
            Val = math.cos(val)

    return Str, Val

These python methods can help develop the equations, just by replicating the formulas. This can be done with a reasonably small piece of code as we are just collecting the data per bus, and plugging the equation.

In [10]:
def develop_PQ_V01(txt, bus1, Ybus, Bus_Data, Prnt=True):

    PQ = get_string(txt, bus1)
    print('%s = ' % PQ, end='')

    V1 = get_string('V', bus1)

    Number_Buses = len(Ybus)
    for bus2 in range(Number_Buses):

        V2 = get_string('V', bus2)
        𝜃12 = get_string('𝜃', bus1, bus2)
        G12 = get_string('G', bus1, bus2)
        B12 = get_string('B', bus1, bus2)

        sc1, _ = get_P_Q(txt, 1)
        sgn, _ = get_P_Q(txt, 2)
        sc2, _ = get_P_Q(txt, 3)

        if Prnt:
            print('%s%s[%s%s%s%s%s%s%s]'
                  % (V1, V2, G12, sc1, 𝜃12, sgn, B12, sc2, 𝜃12), end=' ')
            if bus2 < Number_Buses-1:
                print('+ ', end='')
            else:
                print()

Number_Buses = len(Ybus)
for bus1 in range(Number_Buses):
    develop_PQ_V01('P', bus1, Ybus, Bus_Data)
print()
for bus1 in range(Number_Buses):
    develop_PQ_V01('Q', bus1, Ybus, Bus_Data)

P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₃ = V₃V₁[G₃,₁cos𝜃₃,₁+B₃,₁sin𝜃₃,₁] + V₃V₂[G₃,₂cos𝜃₃,₂+B₃,₂sin𝜃₃,₂] + V₃V₃[G₃,₃cos𝜃₃,₃+B₃,₃sin𝜃₃,₃] 

Q₁ = V₁V₁[G₁,₁sin𝜃₁,₁-B₁,₁cos𝜃₁,₁] + V₁V₂[G₁,₂sin𝜃₁,₂-B₁,₂cos𝜃₁,₂] + V₁V₃[G₁,₃sin𝜃₁,₃-B₁,₃cos𝜃₁,₃] 
Q₂ = V₂V₁[G₂,₁sin𝜃₂,₁-B₂,₁cos𝜃₂,₁] + V₂V₂[G₂,₂sin𝜃₂,₂-B₂,₂cos𝜃₂,₂] + V₂V₃[G₂,₃sin𝜃₂,₃-B₂,₃cos𝜃₂,₃] 
Q₃ = V₃V₁[G₃,₁sin𝜃₃,₁-B₃,₁cos𝜃₃,₁] + V₃V₂[G₃,₂sin𝜃₃,₂-B₃,₂cos𝜃₃,₂] + V₃V₃[G₃,₃sin𝜃₃,₃-B₃,₃cos𝜃₃,₃] 


[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

## Simplifying the equations

To simplify the equations, we can update the equations by removing the conductances ($G$) and susceptances ($B$) that have a value of zero. The same code above is used, but additional lines are included to check the values of the impedances (the model is growing!).

In [11]:
def develop_PQ_V02(txt, bus1, Ybus, Bus_Data, Prnt=True):

    PQ = get_string(txt, bus1)
    if Prnt:
        print('%s = ' % PQ, end='')

    V1 = get_string('V', bus1)

    Gflag = []
    Bflag = []
    flg = False
    for bus2 in range(len(Ybus)):

        # Is G greater than zero?
        if abs(Ybus[bus1][bus2].real) > 0.0001:
            flgg = True
        else:
            flgg = False
        Gflag.append(flgg)

        # Is B is greater than zero zero?
        if abs(Ybus[bus1][bus2].imag) > 0.0001:
            flgb = True
        else:
            flgb = False
        Bflag.append(flgb)

        if flgg or flgb:
            if flg and Prnt:
                print(' + ', end='')
            flg = True
            V2 = get_string('V', bus2)
            𝜃12 = get_string('𝜃', bus1, bus2)
            if Prnt:
                print('%s*%s*' % (V1, V2), end='')

        if flgg and flgb and Prnt:
            print('[', end='')

        if flgg:
            G12 = get_string('G', bus1, bus2)
            sc1, _ = get_P_Q(txt, 1)
            if Prnt:
                print('%s*%s%s' % (G12, sc1, 𝜃12), end='')

        if flgb:
            _, sgnv = get_P_Q(txt, 2)
            if sgnv < 0:
                if Prnt:
                    print('-', end='')
            else:
                if flgg:
                    if Prnt:
                        print('+', end='')
            B12 = get_string('B', bus1, bus2)
            sc2, _ = get_P_Q(txt, 3)
            if Prnt:
                print('%s*%s%s' % (B12, sc2, 𝜃12), end='')

        if flgg and flgb and Prnt:
            print(']', end='')
    if Prnt:
        print()

    return Gflag, Bflag

P_Gflg = []
P_Bflg = []
for bus in range(Number_Buses):
    Gflag, Bflag = develop_PQ_V02('P', bus, Ybus, Bus_Data)
    P_Gflg.append(Gflag)
    P_Bflg.append(Bflag)

print()
Q_Gflg = []
Q_Bflg = []
for bus1 in range(Number_Buses):
    Gflag, Bflag = develop_PQ_V02('Q',bus1, Ybus, Bus_Data)
    Q_Gflg.append(Gflag)
    Q_Bflg.append(Bflag)

P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₂*B₂,₂*sin𝜃₂,₂ + V₂*V₃*B₂,₃*sin𝜃₂,₃
P₃ = V₃*V₁*B₃,₁*sin𝜃₃,₁ + V₃*V₂*B₃,₂*sin𝜃₃,₂ + V₃*V₃*B₃,₃*sin𝜃₃,₃

Q₁ = V₁*V₁*-B₁,₁*cos𝜃₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂ + V₁*V₃*-B₁,₃*cos𝜃₁,₃
Q₂ = V₂*V₁*-B₂,₁*cos𝜃₂,₁ + V₂*V₂*-B₂,₂*cos𝜃₂,₂ + V₂*V₃*-B₂,₃*cos𝜃₂,₃
Q₃ = V₃*V₁*-B₃,₁*cos𝜃₃,₁ + V₃*V₂*-B₃,₂*cos𝜃₃,₂ + V₃*V₃*-B₃,₃*cos𝜃₃,₃


It is possible to further develop the equations considering that:
> $sin(0)=0$ 

> $sin(-𝜃) = - sin(𝜃)$ 

> $cos(0) = 1$.

> $cos(-𝜃) = cos(𝜃)$.

Again, we start with the code above, but more lines are added to account for the trigonometric equation (the code continues to grow!). As the model is becoming a bit big, we are going to collect some outputs (e.g., when to ignore parts of the equaions, etc.) so that the next version of the code will not be as big.

In [12]:
def develop_PQ_V03(txt, bus1, Ybus, Bus_Data, Gflg, Bflg, Prnt=True):

    PQ = get_string(txt, bus1)
    if Prnt:
        print('%s = ' % PQ, end='')

    V1 = get_string('V', bus1)

    Gflag = []
    Bflag = []
    flg = False
    Nxt = [False for x in range(len(Ybus))]
    for bus2 in range(len(Ybus)):

        # Is the first component non-zero?
        flgg = False
        if Gflg[bus2]:
            sc1, scv1 = get_P_Q(txt, 1)
            if bus1 == bus2:
                if scv1 != 0:
                    flgg = True
            else:
                flgg = True
        Gflag.append(flgg)

        # Is the second component non-zero?
        flgb = False
        if Bflg[bus2]:
            sc2, scv2 = get_P_Q(txt, 3)
            if bus1 == bus2:
                if scv2 != 0:
                    flgb = True
            else:
                flgb = True
        Bflag.append(flgb)

        if flgg or flgb:
            if flg and Prnt:
                Nxt[bus2] = True
                print(' + ', end='')
            flg = True
            if bus1 == bus2:
                if Prnt:
                    print('%s\u00b2*' % V1, end='')
            else:
                V2 = get_string('V', bus2)
                𝜃12 = get_string('𝜃', bus1, bus2)
                if Prnt:
                    print('%s*%s*' % (V1, V2), end='')

        if flgg and flgb and Prnt:
            print('[', end='')

        if flgg:
            G12 = get_string('G', bus1, bus2)
            𝜃12 = get_string('𝜃', bus1, bus2)
            if Prnt:
                print('%s*%s%s' % (G12, sc1, 𝜃12), end='')

        if flgb:
            _, sgnv = get_P_Q(txt, 2)
            if sgnv < 0:
                if Prnt:
                    print('-', end='')
            else:
                if flgg and Prnt:
                    print('+', end='')
            B12 = get_string('B', bus1, bus2)
            if Prnt:
                if bus1 == bus2:
                    print('%s' % B12, end='')
                else:
                    print('%s*%s%s' % (B12, sc2, 𝜃12), end='')

        if flgg and flgb and Prnt:
            print(']', end='')

    if Prnt:
        print()

    return Gflag, Bflag, Nxt

P_Gflag = []
P_Bflag = []
P_Next = []
for bus in range(Number_Buses):
    Gflag, Bflag, Nxt = develop_PQ_V03('P', bus, Ybus, Bus_Data, P_Gflg[bus], P_Bflg[bus])
    P_Gflag.append(Gflag)
    P_Bflag.append(Bflag)
    P_Next.append(Nxt)

print()
Q_Gflag = []
Q_Bflag = []
Q_Next = []
for bus in range(Number_Buses):
    Gflag, Bflag, Nxt = develop_PQ_V03('Q', bus, Ybus, Bus_Data, Q_Gflg[bus], Q_Bflg[bus])
    Q_Gflag.append(Gflag)
    Q_Bflag.append(Bflag)
    Q_Next.append(Nxt)

P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₃*B₂,₃*sin𝜃₂,₃
P₃ = V₃*V₁*B₃,₁*sin𝜃₃,₁ + V₃*V₂*B₃,₂*sin𝜃₃,₂

Q₁ = V₁²*-B₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂ + V₁*V₃*-B₁,₃*cos𝜃₁,₃
Q₂ = V₂*V₁*-B₂,₁*cos𝜃₂,₁ + V₂²*-B₂,₂ + V₂*V₃*-B₂,₃*cos𝜃₂,₃
Q₃ = V₃*V₁*-B₃,₁*cos𝜃₃,₁ + V₃*V₂*-B₃,₂*cos𝜃₃,₂ + V₃²*-B₃,₃


[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

## Considering known values

Let us continue simplifying the equations, now considering the known values.

This time, we will use the data produced from the previous method and, for the sake of convenience, a supplementary method is created to manage the trigonometric functions.

In [13]:
def get_Bus_Value(Bus_Data, txt, bus):
    '''Get known and unknown data'''

    # Is the data known?
    if txt in Bus_Data[bus].keys():
        Val = Bus_Data[bus][txt]  # Get value
        Str1 = str(Val)  # Get string (number of symbol)
        Str2 = ''  # Get string (number of empty)
        flg = True
    else:
        Str1 = get_string(txt, bus)  # Get string (number of symbol)
        Str2 = Str1  # Get string (number of empty)
        Val = 1
        flg = False

    return Str1, Str2, Val, flg

The equations are now updated by replacing the variables with known values. This time, thanks for the data we collected, and the method above, the new piece of code is not as big as before as it only includes the new parts.

In [14]:
def develop_PQ_V04(txt, bus1, Ybus, Bus_Data, Gflg, Bflg, Nxt):

    PQ, _, _, _ = get_Bus_Value(Bus_Data, txt, bus1)
    print('%s = ' % PQ, end='')

    V1, _, _, _ = get_Bus_Value(Bus_Data, 'V', bus1)
    𝜃1, _, _, _ = get_Bus_Value(Bus_Data, '𝜃', bus1)

    for bus2 in range(len(Ybus)):
        flgg = Gflg[bus2]
        flgb = Bflg[bus2]

        if flgg or flgb:
            if Nxt[bus2]:
                print(' + ', end='')

            if bus1 == bus2:
                print('%s\u00b2*' % V1, end='')
            else:
                V2, _, _, _ = get_Bus_Value(Bus_Data, 'V', bus2)
                𝜃2, _, _, _ = get_Bus_Value(Bus_Data, '𝜃', bus2)
                print('%s*%s*' % (V1, V2), end='')

        if flgg and flgb:
            print('[', end='')

        if flgg:
            G12 = str(Ybus[bus1][bus2].real)
            𝜃2, _, _, _ = get_Bus_Value(Bus_Data, '𝜃', bus2)
            sc1, _ = get_P_Q(txt, 1)
            print('%s*%s(%s-%s)' % (G12, sc1, 𝜃1, 𝜃2), end='')

        if flgb:
            _, sgnv = get_P_Q(txt, 2)
            if sgnv < 0:
                print('-', end='')
            else:
                if flgg:
                    print('+', end='')
            B12 = str(Ybus[bus1][bus2].imag)
            if bus1 == bus2:
                print('%s' % B12, end='')
            else:
                sc2, _ = get_P_Q(txt, 3)
                print('%s*%s(%s-%s)' % (B12, sc2, 𝜃1, 𝜃2), end='')

        if flgg and flgb:
            print(']', end='')

    print()

for bus in range(Number_Buses):
    develop_PQ_V04('P', bus, Ybus, Bus_Data, P_Gflag[bus], P_Bflag[bus], P_Next[bus])
print()
for bus in range(Number_Buses):
    develop_PQ_V04('Q', bus, Ybus, Bus_Data, Q_Gflag[bus], Q_Bflag[bus], Q_Next[bus])

P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
-1.5 = V₂*1*10.0*sin(𝜃₂-0) + V₂*1*4.0*sin(𝜃₂-𝜃₃)
1 = 1*1*5.0*sin(𝜃₃-0) + 1*V₂*4.0*sin(𝜃₃-𝜃₂)

Q₁ = 1²*--15.0 + 1*V₂*-10.0*cos(0-𝜃₂) + 1*1*-5.0*cos(0-𝜃₃)
-0.8 = V₂*1*-10.0*cos(𝜃₂-0) + V₂²*--14.0 + V₂*1*-4.0*cos(𝜃₂-𝜃₃)
Q₃ = 1*1*-5.0*cos(𝜃₃-0) + 1*V₂*-4.0*cos(𝜃₃-𝜃₂) + 1²*--9.0


In [15]:
def develop_𝜃(txt, Bus_Data, No, bus1, bus2, 𝜃s1, 𝜃v1, 𝜃f1, 𝜃s2, 𝜃v2, 𝜃f2):
    if bus1 == bus2:
        _, Val = get_P_Q(txt, No, 0)
        Str1 = ''
        Str = None
    elif 𝜃f1 and 𝜃f2:
        _, Val = get_P_Q(txt, No, 𝜃v1-𝜃v2)
        Str1 = ''
        Str = None
    elif 𝜃f1:
        Str, Val = get_P_Q(txt, No, -1)
        if 𝜃v1 == 0:
            if Val < 0:
                Val = -1
            else:
                Val = 1
            Str1 = Str + '(' + 𝜃s2 + ')'
        else:
            Str1 = Str + '(' + str(𝜃v1) + '-' + 𝜃s2 + ')'
    elif 𝜃f2:
        Str, _ = get_P_Q(txt, No)
        Val = 1
        if 𝜃v2 == 0:
            Str1 = Str + '(' + 𝜃s1 + ')'
        else:
            print('Part 2', 𝜃s2, 𝜃v2, 𝜃f2)
            Str1 = Str + '(' + 𝜃s1 + '-' + str(𝜃v2) + ')'
    else:
        Str, _ = get_P_Q(txt, No)
        Val = 1
        Str1 = Str + '(' + 𝜃s1 + '-' + 𝜃s2 + ')'

    return Val, Str1, Str

def develop_PQ_V05(txt, bus1, Ybus, Bus_Data, Gflg, Bflg, Nxt, flg=False,
                   Prnt=True):

    PQ, _, PQv, PQf = get_Bus_Value(Bus_Data, txt, bus1)
    if Prnt:
        print('%s = ' % PQ, end='')
    if PQf:
        Delta_Data = PQv
    else:
        Delta_Data = None

    _, Vs1, Vv1, Vf1 = get_Bus_Value(Bus_Data, 'V', bus1)
    𝜃1, 𝜃s1, 𝜃v1, 𝜃f1 = get_Bus_Value(Bus_Data, '𝜃', bus1)

    PQ_Data = []
    for bus2 in range(len(Ybus)):
        PQ_Dt = {'V1': None, 'V2': None,
                 'scA': None, '𝜃A1': None, '𝜃A2': None,
                 'scB': None, '𝜃B1': None, '𝜃B2': None,
                 'val1': None, 'val2': None, 'val3': None}

        flgg = Gflg[bus2]
        flgb = Bflg[bus2]

        _, Vs2, Vv2, Vf2 = get_Bus_Value(Bus_Data, 'V', bus2)
        𝜃2, 𝜃s2, 𝜃v2, 𝜃f2 = get_Bus_Value(Bus_Data, '𝜃', bus2)

        if bus1 == bus2 and Vs1 != '':
            Vs12 = Vs1 + '\u00b2'
        else:
            Vs12 = Vs1+Vs2

        scv1, scs1, sc1 = develop_𝜃(txt, Bus_Data, 1, bus1, bus2, 𝜃s1, 𝜃v1,
                                    𝜃f1, 𝜃s2, 𝜃v2, 𝜃f2)
        scv2, scs2, sc2 = develop_𝜃(txt, Bus_Data, 3, bus1, bus2, 𝜃s1, 𝜃v1,
                                    𝜃f1, 𝜃s2, 𝜃v2, 𝜃f2)

        _, sgnv = get_P_Q(txt, 2)

        # Should both G and B components be considered?
        if flgg and flgb:
            if Nxt[bus2] and Prnt:
                print(' + ', end='')
            PQ_Dt['val1'] = Vv1*Vv2
            if Prnt:
                print('%.4f%s[' % (PQ_Dt['val1'], Vs12), end='')

            PQ_Dt['val2'] = scv1*Ybus[bus1][bus2].real
            PQ_Dt['scA'] = sc1
            if Prnt:
                print('%.4f%s' % (PQ_Dt['val2'], scs1), end='')

            PQ_Dt['val3'] = sgnv*scv2*Ybus[bus1][bus2].imag
            if PQ_Dt['val3'] > 0 and Prnt:
                print('+', end='')

            PQ_Dt['scB'] = sc2
            if Prnt:
                print('%.4f%s' % (PQ_Dt['val3'], scs2), end='')
                print(']', end='')

        elif flgg:  # Should only G be considered?
            PQ_Dt['val1'] = Vv1*Vv2*scv1*Ybus[bus1][bus2].real
            if Nxt[bus2] and Nxt[bus2] > 0 and Prnt:
                print(' + ', end='')
            PQ_Dt['scA'] = sc1
            if Prnt:
                print('%.4f%s' % (PQ_Dt['val1'], Vs12+scs1), end='')

        elif flgb:  # Should only B be considered
            PQ_Dt['val1'] = Vv1*Vv2*scv2*sgnv*Ybus[bus1][bus2].imag
            if Nxt[bus2] and PQ_Dt['val1'] > 0 and Prnt:
                print(' + ', end='')
            PQ_Dt['scA'] = sc2
            if Prnt:
                print('%.4f%s' % (PQ_Dt['val1'], Vs12+scs2), end='')

        if flgg or flgb:
            if not Vf1:
                PQ_Dt['V1'] = bus1
                if not Vf2:
                    PQ_Dt['V2'] = bus2
            else:
                if not Vf2:
                    PQ_Dt['V1'] = bus2

            aux = '𝜃A'
            if flgg and scs1 != '':
                if not 𝜃f1:
                    PQ_Dt['𝜃A1'] = bus1
                    aux = '𝜃B'
                    if not 𝜃f2:
                        PQ_Dt['𝜃A2'] = bus2
                else:
                    if not 𝜃f2:
                        PQ_Dt['𝜃A1'] = bus2
                        aux = '𝜃B'

            if flgb and scs2 != '':
                if not 𝜃f1:
                    PQ_Dt[aux+'1'] = bus1
                    if not 𝜃f2:
                        PQ_Dt[aux+'2'] = bus2
                else:
                    if not 𝜃f2:
                        PQ_Dt[aux+'1'] = bus2

            PQ_Data.append(PQ_Dt)
    if Prnt:
        if PQf:
            print('.....[Implicit]')
        else:
            print('.....[Explicit]')

    if flg:
        return (PQ_Data, Delta_Data, bus1, txt)

for bus in range(Number_Buses):
    develop_PQ_V05('P', bus, Ybus, Bus_Data, P_Gflag[bus], P_Bflag[bus], P_Next[bus])
print()
for bus in range(Number_Buses):
    develop_PQ_V05('Q', bus, Ybus, Bus_Data, Q_Gflag[bus], Q_Bflag[bus], Q_Next[bus])

P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]
-1.5 = 10.0000V₂sin(𝜃₂) + 4.0000V₂sin(𝜃₂-𝜃₃).....[Implicit]
1 = 5.0000sin(𝜃₃) + 4.0000V₂sin(𝜃₃-𝜃₂).....[Implicit]

Q₁ = 15.0000-10.0000V₂cos(𝜃₂)-5.0000cos(𝜃₃).....[Explicit]
-0.8 = -10.0000V₂cos(𝜃₂) + 14.0000V₂²-4.0000V₂cos(𝜃₂-𝜃₃).....[Implicit]
Q₃ = -5.0000cos(𝜃₃)-4.0000V₂cos(𝜃₃-𝜃₂) + 9.0000.....[Explicit]


[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

## Developing as a tool

You may not be interested in what is inside the model, but rather on using it as a tool. To cater for that need, the code developed above is added to a single method, which is called `develop_PF_Equations`.

In [16]:
def develop_PF_Equations(Load, Generator, Ybus, flg=False, Prnt=True):
    import math
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)
    if Prnt:
        print()
        display_Bus_Type(Bus_Data, Bus_Type)
    if Prnt:
        print()
        print('***DEVELOP AND SIMPLIFY***')
    Number_Buses = len(Ybus)

    P_Data = []
    for bus in range(Number_Buses):
        if Prnt:
            print('P%d:'%(bus+1))
            develop_PQ_V01('P', bus, Ybus, Bus_Data)
        Gflag, Bflag = develop_PQ_V02('P',bus, Ybus, Bus_Data, Prnt)
        Gflag, Bflag, Nxt = develop_PQ_V03('P', bus, Ybus, Bus_Data, Gflag, Bflag, Prnt)
        if Prnt:
            develop_PQ_V04('P', bus, Ybus, Bus_Data, Gflag, Bflag, Nxt)
        P_Dt = develop_PQ_V05('P', bus, Ybus, Bus_Data, Gflag, Bflag, Nxt, flg, Prnt)
        P_Data.append(P_Dt)
        if Prnt:
            print()

    Q_Data = []
    for bus in range(Number_Buses):
        if Prnt:
            print('Q%d:'%(bus+1))
            develop_PQ_V01('Q', bus, Ybus, Bus_Data)
        Gflag, Bflag = develop_PQ_V02('Q',bus, Ybus, Bus_Data, Prnt)
        Gflag, Bflag, Nxt = develop_PQ_V03('Q', bus, Ybus, Bus_Data, Gflag, Bflag, Prnt)
        if Prnt:
            develop_PQ_V04('Q', bus, Ybus, Bus_Data, Gflag, Bflag, Nxt)
        Q_Dt = develop_PQ_V05('Q', bus, Ybus, Bus_Data, Gflag, Bflag, Nxt, flg, Prnt)
        Q_Data.append(Q_Dt)
        if Prnt:
            print()

    if flg:
        return P_Data, Q_Data

Now that the method has been created, the step-by-step development of the power flow equations can be simulated as follows:

In [17]:
develop_PF_Equations(Load, Generator, Ybus)


Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₂*B₂,₂*sin𝜃₂,₂ + V₂*V₃*B₂,₃*sin𝜃₂,₃
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₃*B₂,₃*sin𝜃₂,₃
-1.5 = V₂*1*10.0*sin(𝜃₂-0) + V₂*1*4.0*sin(𝜃₂-𝜃₃)
-1.5 = 10.0000V₂sin(𝜃₂) + 4.0000V₂sin(𝜃₂-𝜃₃).....[Implicit]

P3:
P₃ = V₃V₁[G₃,₁cos𝜃₃,₁+B₃,₁sin𝜃₃,₁] + V₃V₂[G₃,₂cos𝜃₃,₂+B₃,₂sin𝜃₃,₂] + V₃V₃[G₃,₃cos𝜃₃,₃+B₃,₃sin𝜃₃,₃] 
P₃ = V₃*V₁*B₃,₁*sin𝜃₃,₁ + V₃*

You can now use this new tool to get detailed step-by-step examples of the development and simplification of the power flow equations.

[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

## Case studies

### Three-bus network

The example presented above can be solved with the developed tool as follows:

<img src="Figures/Week02_3Bus.png" alt="Fig01" class="bg-primary" width="500px">

In [18]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]
Load = [
    [2, complex(1.5, 0.8)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1}
]
Ybus = get_Ybus(Connectivity, True)
develop_PF_Equations(Load, Generator, Ybus)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₂ = V₂*V₁*

[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

### Two-bus network

Let us now try a different example, a 2-bus system:

![Week02_2Bus.png](Figures/Week02_2Bus.png)

In [19]:
Connectivity = [
    [1, 2, complex(0, 0.10)]
]
Load = [
    [2, complex(0.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 }
]
Ybus = get_Ybus(Connectivity, True)
develop_PF_Equations(Load, Generator, Ybus)

The network has 1 branches and 2 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
_______|___________|__________

Ybus = 
 [[ 0.-10.j -0.+10.j]
 [-0.+10.j  0.-10.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -0.5000   -0.5000

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂
P₁ = 1*V₂*10.0*sin(0-𝜃₂)
P₁ = -10.0000V₂sin(𝜃₂).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] 
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₂*B₂,₂*sin𝜃₂,₂
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁
-0.5 = V₂*1*10.0*sin(𝜃₂-0)
-0.5 = 10.0000V₂sin(𝜃₂).....[Implicit]

Q1:
Q₁ = V₁V₁[G₁,₁sin𝜃₁,₁-B₁,₁cos𝜃₁,₁] + V₁V₂[G₁,₂sin𝜃₁,₂-B₁,₂cos𝜃₁,₂] 
Q₁ = V₁*V₁*-B₁,₁*cos𝜃₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂
Q₁ = V₁²*-B₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂
Q₁ = 1²*--

[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

### Four-bus network

Let us now try a bigger network:

![Week02_4Bus.png](Figures/Week02_4Bus.png)

In [20]:
Connectivity = [
    [1, 2, complex(0, 0.25)],
    [1, 3, complex(0, 0.5)],
    [3, 4, complex(0, 0.25)]
]
Load = [
    [2, complex(0.5, 0.3)],
    [3, complex(1, 0.5)],
    [4, complex(1, 0.5)],
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'V':1, 'P':1.2 }
]
Ybus = get_Ybus(Connectivity, True)
develop_PF_Equations(Load, Generator, Ybus)

The network has 3 branches and 4 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.25j
    2  |    1 -  3 | 0.5j
    3  |    3 -  4 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-6.j -0.+4.j -0.+2.j  0.+0.j]
 [-0.+4.j  0.-4.j  0.+0.j  0.+0.j]
 [-0.+2.j  0.+0.j  0.-6.j -0.+4.j]
 [ 0.+0.j  0.+0.j -0.+4.j  0.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -0.5000   -0.3000
   3     PV   1.0000        ?    0.2000         ?
   4     PQ        ?        ?   -1.0000   -0.5000

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] + V₁V₄[G₁,₄cos𝜃₁,₄+B₁,₄sin𝜃₁,₄] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*4.0*sin(0-𝜃₂) + 1*1*2.0*sin(0-𝜃₃)
P₁ = -4.0000V₂sin(𝜃₂)-2.0000sin(𝜃₃).

[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

### Three-bus radial network

We can also develop the equations for networks with complex impedances:

![Week02_2Bus_radial.png](Figures/Week02_2Bus_radial.png)

In [21]:
Connectivity = [
    [1, 2, complex(0.1, 0.2)],
    [2, 3, complex(0.1, 0.2)]
]
Load = [
    [2, complex(1.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1.05, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1 }
]
Ybus = get_Ybus(Connectivity, True)
develop_PF_Equations(Load, Generator, Ybus)

The network has 2 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0.2j)
    2  |    2 -  3 | (0.1+0.2j)
_______|___________|__________

Ybus = 
 [[ 2.-4.j -2.+4.j  0.+0.j]
 [-2.+4.j  4.-8.j -2.+4.j]
 [ 0.+0.j -2.+4.j  2.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0500   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.5000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*[G₁,₁*cos𝜃₁,₁+B₁,₁*sin𝜃₁,₁] + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = V₁²*G₁,₁*cos𝜃₁,₁ + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = 1.05²*2.0*cos(0-0) + 1.05*V₂*[-2.0*cos(0-𝜃₂)+4.0*sin(0-𝜃₂)]
P₁ = 2.2050 + 1.0500V₂[-2.0000cos(𝜃₂)-4.0000sin(𝜃₂)].....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,

[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)

### Create your own example

Try to create your own examples:

In [22]:
Connectivity = [
    [1, 2, complex(0.1, 0.2)],
    [2, 3, complex(0.1, 0.2)]
]
Load = [
    [2, complex(1.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1.05, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1 }
]
Ybus = get_Ybus(Connectivity, True)
develop_PF_Equations(Load, Generator, Ybus)

The network has 2 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0.2j)
    2  |    2 -  3 | (0.1+0.2j)
_______|___________|__________

Ybus = 
 [[ 2.-4.j -2.+4.j  0.+0.j]
 [-2.+4.j  4.-8.j -2.+4.j]
 [ 0.+0.j -2.+4.j  2.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0500   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.5000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*[G₁,₁*cos𝜃₁,₁+B₁,₁*sin𝜃₁,₁] + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = V₁²*G₁,₁*cos𝜃₁,₁ + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = 1.05²*2.0*cos(0-0) + 1.05*V₂*[-2.0*cos(0-𝜃₂)+4.0*sin(0-𝜃₂)]
P₁ = 2.2050 + 1.0500V₂[-2.0000cos(𝜃₂)-4.0000sin(𝜃₂)].....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,

[Back to top](#EEEN30131-Power-System-Analysis:-Week-02---Power-Flow-Formulation)